In [1]:
# -*- coding: utf-8 -*-
"""

XXX brouillon XXX
pointillé noir = rayons correspondant à un objet sur l'axe à la position 
vérifiant les relations de conjugaison

rayons bleus : correspondent à l'objet le plus loin net
rayons rouges : correspondent à l'objet le plus proche net


Les calculs sont fait dans le cadre de l'approximation des petits angles, ne 
tient pas compte du côté non stigmatique des lentilles, en particulier aux 
grandes ouvertures. Calculs non garantis car fait vite fait sur un brouillon.
(Basé sur le formalisme de l'optique matricielle, plus facile qu'avec les 
relations de conjugaison et de la trigo)

Ne tiens pas compte non plus du phénomène de diffraction qui est le phénomène
(à ma connaissance) qui limite la résolution lorsque les diaphragmes sont très
fermés.

"""
%matplotlib inline
import numpy as np #testé avec np.__version__ 1.16.1
import matplotlib.pyplot as plt #testé avec matplotlib.__version__ 2.0.0
import PIL.Image as im#attention à la majuscule ; testé avec PIL.__version__ '5.4.1'
from matplotlib.widgets import Slider, Button#, RadioButtons
from scipy.signal import fftconvolve #testé avec scipy.__version__ 1.2.1

# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '150px'}
slider_layout = Layout(width='99%')

def traverse_vide(x_old,y_old,theta_old,x_new):
    """ optique matricielle 
            traversé du vide : theta inchangé, y change : ynew = yold + theta*dist
    """
    return y_old + (x_new-x_old)*theta_old #theta_new = theta_old
def traverse_lentille(y_old,theta_old,fp):
    """ optique matricielle 
            traversé d'une lentille : y inchangé, theta_new = theta_old - y/f'
    """
    theta_new = -y_old/fp + theta_old
    return theta_new #y_new = y_old

def traverse_lentille_arrière(y_new, theta_new, fp):
    """ optique matricielle 
            traversé d'une lentille : y inchangé, theta_new = theta_old - y/f'
            ici, on connait theta_new et on veut theta_old
    """
    theta_old = theta_new + y_new/fp
    return theta_old


def main(xAp,R,epsi): # fonction de tracé qui sera actualisée
    
    ####### définition des constante et valeurs initiales
    fp = 1 #distance focale de la lentille
    #xAp = 1.5 #position algébrique du capteur
    #epsi = 0.5 #taille(rayon) du «cercle de confusion
    #R = 2.3 #rayon du diaphragme

    inf = float("inf")
    #####   limite de l'espace  pour matplotlib  #####
    xA = 1/(-1/fp+1/xAp) #position de l'image
    lim = [2*xA,xAp*2,-4,4]   # xdeb xfin ydeb yfin
    lim =[-5,5,-4,4] # LIMITES DE L'ESPACE = VALEURS FIXES
    marge = 0.05
    x_deb = lim[0]
    x_fin = lim[1]
    y_deb = lim[2]
    y_fin = lim[3]
    delta_x = x_fin - x_deb
    delta_y = y_fin - y_deb
    ###### fin de déf des constantes

    fig = plt.figure(figsize = [13,10])

    #axe optique
    plt.plot([x_deb,x_fin],[0,0],'-k')#axe optique
    deb_fleche = x_fin - 0.025*delta_x
    altitude_fleche = delta_y * 0.025
    plt.plot( [deb_fleche,x_fin,deb_fleche],[altitude_fleche,0,-altitude_fleche],'-k') # flèche

    #lentille
    x_O, f_p = 0, fp #une seule lentille ici
    plt.plot([x_O,x_O],[0.995*y_deb,0.995*y_fin],'-k',lw=2)
    if f_p<0:
        sg = -1
    else:
        sg = 1
    lx_fl = 0.05*delta_x
    ly_fl = sg * 0.05*delta_y
    plt.plot([x_O-lx_fl,x_O,x_O+lx_fl],[0.995*y_fin-ly_fl,0.995*y_fin,0.995*y_fin-ly_fl],'-k',lw=2)
    plt.plot([x_O-lx_fl,x_O,x_O+lx_fl],[0.995*y_deb+ly_fl,0.995*y_deb,0.995*y_deb+ly_fl],'-k',lw=2)

    #foyers
    plt.plot([x_O+f_p]*2,[delta_y*0.01,-delta_y*0.01])
    plt.plot([x_O-f_p]*2,[delta_y*0.01,-delta_y*0.01])
    plt.axis('off')
    plt.axis(lim)# limites fixes [-5,3.4,-4,4]

    #cercle de confusion ; position xAp
    plot_cercleConf, = plt.plot([xAp]*2,[-epsi, epsi],lw = 2,color = "red")

    #tracé du diaphragme
    global diaphragme_bas, diaphragme_haut
    diaphragme_bas, = plt.plot([-0.075,-0.075],[0.9*y_deb, -R],'-',lw = 5, color = [0.5,0.5,0.5])
    diaphragme_haut, =plt.plot([-0.075,-0.075],[0.9*y_fin, R],'-',lw = 5, color = [0.5,0.5,0.5])

    #rayons limites = rayon qui passe par le haut du diaphragme + haut et bas du cercle de confusion (et symétrique)
    theta_lim_new1 = (epsi-R)/xAp
    theta_lim_new2 = (-epsi-R)/xAp
    theta_lim_old1 = traverse_lentille_arrière(R,theta_lim_new1,fp)
    theta_lim_old2 = traverse_lentille_arrière(R,theta_lim_new2,fp)
    x_old1 = -R/theta_lim_old1#point sur l'axe de départ pour le rayon
    x_old2 = -R/theta_lim_old2
    #XXX remarque, si x_old1>0, c'est qu'on a atteint l'hyperfocale => x_old1 réel = -inf

    plotlim1_haut, = plt.plot([x_old1,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new1,x_fin)],"-r")
    plotlim1_bas, = plt.plot([x_old1,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new1,x_fin)],"-r")

    plotlim2_haut, = plt.plot([x_old2,0,x_fin],[0,R,traverse_vide(0,R,theta_lim_new2,x_fin)],"-b")
    plotlim2_bas, = plt.plot([x_old2,0,x_fin],[0,-R,-traverse_vide(0,R,theta_lim_new2,x_fin)],"-b")

    #netteté «parfaite»
    plotnet_haut, = plt.plot([xA,0,x_fin],[0,R,traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)],"--k")
    plotnet_bas, =plt.plot([xA,0,x_fin],[0,-R,-traverse_vide(0,R,traverse_lentille(R,-R/xA,fp),x_fin)],"--k")
    profondeur = x_old1-x_old2
    plt.show()
    if profondeur < 0:
        print("profondeur de champ infini")
    else:
        print("profondeur de champ =",profondeur)
    if x_old2 < 0:
        print("dont",int(round(100*(x_old1-xA)/profondeur)),"% proche")
        print("et",int(round(-100*(x_old2-xA)/profondeur)),"% loin")

interact(main, xAp=FloatSlider(min=1.01, max=3, step=0.1, value=1.67, description="xA'", style=style, layout=slider_layout),
               R=FloatSlider(min=0, max=3, step=0.1, value=1.6, description='R', style=style, layout=slider_layout),
               epsi=FloatSlider(min=0, max=1.2, step=0.05, value=0.4, description='epsilon', style=style, layout=slider_layout)
        );








interactive(children=(FloatSlider(value=1.67, description="xA'", layout=Layout(width='99%'), max=3.0, min=1.01…